## Import

In [49]:
import math
import random
import numpy as np
from numpy import asarray
import cv2
from PIL import Image, ImageDraw
import sys
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import pickle
import sys

## Preprocessing and obtain the data coord of many shapes from the txt file


In [50]:
file_1 = open("0_2.txt")
Data= file_1.readlines()
Coordinates=list()
count=0
Shapes=dict()
data_list_coordinates = []
for i in range(len(Data)):
    if "End" in Data[i]:
      Shapes[i]=Data[i-1]
for i in range(len(Data)):
    if "End" in Data[i]:
      Coordinates.append(Data[i-1])
for j in range(len(Coordinates)):
  data_list = Coordinates[j].split(" ")
  data_list_coordinates.append("Shape")
  for item in data_list:
    (x, y) = item.split(",")
    (x, y) = (math.ceil(float(x))), (math.ceil(float(y))+j*400)
    data_list_coordinates.append((x, y))

In [13]:
polygon=['Shape',
 (604, 260),
 (605, 247),
 (608, 245),
 (609, 242),
 (606, 241),
 (598, 238),
 (585, 242),
 (559, 222),
 (367, 309),
 (367, 309),
 (367, 309),
 (367, 309),
 (379, 352),
 (382, 376),
 (388, 490),
 (387, 509),
 (401, 578),
 (401, 577),
 (403, 576),
 (404, 573),
 (407, 573),
 (411, 571),
 (417, 570),
 (430, 568),
 (447, 564),
 (471, 558),
 (500, 551),
 (536, 544),
 (648, 529),
 (679, 526),
 (810, 522),
 (811, 521),
 (811, 521),
 (812, 521),
 (811, 521),
 (812, 521),
 (811, 519),
 (809, 516),
 (804, 512),
 (799, 501),
 (796, 482),
 (793, 464),
 (788, 445),
 (784, 428),
 (780, 409),
 (776, 394),
 (760, 358),
 (760, 336),
 (755, 329),
 (754, 318),
 (750, 313),
 (750, 304),
 (746, 301),
 (745, 294),
 (742, 292),
 (743, 288),
 (741, 288),
 (742, 285),
 (741, 285),
 (741, 285),
 (741, 284),
 (742, 284),
 (742, 285),
 (742, 285),
 (741, 284),
 (737, 282),
 (497, 216),
 (498, 217),
 (499, 219),
 (499, 220),
 (499, 219),
 (499, 219),
 (499, 219),
 (499, 219),
 (499, 219),
 (499, 219),
 (499, 219),
 (499, 218)]

In [14]:
len(Coordinates)

2

In [51]:
data_list_coordinates+=polygon
data_list_coordinates

['Shape',
 (1009, 445),
 (1006, 446),
 (1065, 462),
 (1073, 462),
 (1089, 456),
 (1105, 453),
 (1213, 453),
 (1218, 456),
 (1223, 461),
 (1234, 467),
 (1238, 470),
 (1243, 472),
 (1246, 474),
 (1250, 474),
 (1258, 471),
 (1259, 468),
 (1260, 468),
 (1260, 466),
 (1261, 465),
 (1261, 464),
 (1260, 464),
 (1259, 462),
 (1258, 461),
 (1256, 460),
 (1256, 457),
 (1254, 456),
 (1251, 454),
 (1250, 451),
 (1248, 448),
 (1244, 442),
 (1240, 439),
 (1237, 435),
 (1231, 408),
 (1226, 385),
 (1203, 335),
 (1199, 323),
 (1193, 314),
 (1188, 304),
 (1182, 294),
 (1176, 284),
 (1160, 250),
 (1149, 231),
 (1138, 210),
 (1132, 199),
 (1107, 245),
 (1104, 254),
 (1086, 304),
 (1082, 313),
 (1024, 423),
 (1020, 430),
 (1005, 462),
 (1006, 461),
 'Shape',
 (1384, 717),
 (1384, 716),
 (1382, 714),
 (1378, 713),
 (1369, 711),
 (1358, 714),
 (1346, 715),
 (1332, 720),
 (1092, 721),
 (1088, 724),
 (1083, 733),
 (1082, 735),
 (1090, 769),
 (1094, 781),
 (1109, 839),
 (1113, 857),
 (1117, 874),
 (1121, 891),


# Construct binary image using coords

In [52]:
my_screen_width = 1920
my_screen_height = 1080
length=len(data_list_coordinates)
K="Shape"
res1=[]
res2=[]
number_of_shapes=[data_list_coordinates.index(i,1) for i in data_list_coordinates[1:] if i == K]
number_of_shapes.insert(0,1)
res1=[i for i,x in enumerate(data_list_coordinates) if x==K]
res2=[i for i in range(len(data_list_coordinates)) if data_list_coordinates[i]==K]
#number_of_shapes.insert(len(number_of_shapes),length)
# let's create a 6 x 6 matrix with all pixels in black color
img = np.zeros((my_screen_width, my_screen_height), np.uint8)
data_list_coordinates1=[i for i in data_list_coordinates if i != K]
for data in data_list_coordinates1:
    img[data[0], data[1]] = 255

cv2.imwrite("t1.png", img)

True

# Mirroring

In [53]:
# load the image, create the mirrored image, and the result placeholder
img = Image.open("t1.png")
mirror = img.transpose(Image.FLIP_LEFT_RIGHT).transpose(Image.ROTATE_90)
mirror.save("t1_mirrored.png")

# Connect points using a thick line

In [54]:
# from google.colab.patches import cv2_imshow
img = cv2.imread("t1_mirrored.png")
for z in range(len(res1)):
  (pre_x, pre_y) = data_list_coordinates[res1[z]+1]
  if z != (len(res1)-1):
    for (x, y) in data_list_coordinates[res1[z]+1:res1[z+1]]:
      img = cv2.line(img, (pre_x, pre_y), (x, y), (255, 255, 255), 20)
      (pre_x, pre_y) = (x, y)
  else:
    for (x, y) in data_list_coordinates[res1[z]+1:]:
      img = cv2.line(img, (pre_x, pre_y), (x, y), (255, 255, 255), 20)
      (pre_x, pre_y) = (x, y)
# save our image as a "png" image
# cv2_imshow(img)
cv2.imwrite("t2.png", img)

True

## Cropping Using Contours

In [55]:
img_orig = cv2.imread('t2.png', 0)

mask = np.zeros(img_orig.shape, np.uint8)  # mask image the final image without small pieces

# using findContours func to find the none-zero pieces
contours, hierarchy = cv2.findContours(img_orig, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
count=0
# draw the white paper and eliminate the small pieces (less than 1000000 px). This px count is the same as the QR code dectection
index = 0
for cnt in contours:
    if cv2.contourArea(cnt) > 100:
        cv2.drawContours(mask, [cnt], 0, 255,-1)  
        # the [] around cnt and 3rd argument 0 mean only the particular contour is drawn

        # Build a ROI to crop the QR
        x, y, w, h = cv2.boundingRect(cnt)
        roi = mask[y:y + h, x:x + w]
        # crop the original QR based on the ROI
        img_crop = img_orig[y:y + h, x:x + w]
        # use cropped mask image (roi) to get rid of all small pieces
        img_final = img_crop * (roi / 255)
        count+=1
        cv2.imwrite('t2_cropped'+str(count)+'.png', img_final)

#cv2.imwrite('t2_cropped.png', img_final)

In [56]:
img = cv2.imread('t2.PNG')
imgGry = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret , thrash = cv2.threshold(imgGry, 240 , 255, cv2.CHAIN_APPROX_NONE)
contours , hierarchy = cv2.findContours(thrash, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)



for contour in contours:
    approx = cv2.approxPolyDP(contour, 0.01* cv2.arcLength(contour, True), True)
    cv2.drawContours(img, [approx], 0, (255, 0, 0), 5)
    x = approx.ravel()[0]
    y = approx.ravel()[1] - 5
    if len(approx) == 3:
        cv2.putText( img, "Triangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 5, (255, 255, 255) )
    elif len(approx) == 4 :
        x, y , w, h = cv2.boundingRect(approx)
        aspectRatio = float(w)/h
        print(aspectRatio)
        if aspectRatio >= 0.95 and aspectRatio < 1.05:
            cv2.putText(img, "square", (x, y), cv2.FONT_HERSHEY_COMPLEX, 5, (255, 255, 255))

        else:
            cv2.putText(img, "rectangle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 5, (255, 255, 255))

    elif len(approx) == 5 :
        cv2.putText(img, "pentagon", (x, y), cv2.FONT_HERSHEY_COMPLEX, 5, (255, 255, 255))
    elif len(approx) == 10 :
        cv2.putText(img, "star", (x, y), cv2.FONT_HERSHEY_COMPLEX, 5, (255, 255, 255))
    else:
        cv2.putText(img, "circle", (x, y), cv2.FONT_HERSHEY_COMPLEX, 5, (255, 255, 255))

cv2.imwrite("t2Final.png", img)

1.7777777777777777


True

## testing cell

In [32]:
#res1
#count
len(contours)

7